In [235]:
import pandas as pd
import utils
import instrument
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

In [245]:
pair = 'CAD_CHF'
granularity = 'H1'
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [246]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [247]:
#calculating and plotting moving average
df_ma = df[['time','mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [248]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-04-28T20:00:00.000000000Z,0.75904,0.75919,0.75870,0.75891,0.757534,0.754293
1,2022-04-28T21:00:00.000000000Z,0.75886,0.75915,0.75862,0.75898,0.757769,0.754357
2,2022-04-28T22:00:00.000000000Z,0.75898,0.75908,0.75863,0.75896,0.757869,0.754430
3,2022-04-28T23:00:00.000000000Z,0.75900,0.75918,0.75872,0.75894,0.758030,0.754526
4,2022-04-29T00:00:00.000000000Z,0.75894,0.75970,0.75868,0.75970,0.758145,0.754623
5,2022-04-29T01:00:00.000000000Z,0.75966,0.76032,0.75956,0.75964,0.758308,0.754738
6,2022-04-29T02:00:00.000000000Z,0.75964,0.76013,0.75955,0.76013,0.758546,0.754863
7,2022-04-29T03:00:00.000000000Z,0.76010,0.76045,0.76002,0.76022,0.758796,0.754996
8,2022-04-29T04:00:00.000000000Z,0.76022,0.76110,0.76016,0.76098,0.758963,0.755182
9,2022-04-29T05:00:00.000000000Z,0.76098,0.76128,0.76053,0.76120,0.759247,0.755394


In [249]:
#Identifying Moving Average using Cross Strategy
#Buy = 1
#Sell = -1
#Negative DELTA Value = GAIN
#Positive DELTA Value = LOSS

In [250]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [251]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [252]:
df_trades['DELTA'] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades['GAIN'] = df_trades['DELTA'] * df_trades['IS_TRADE']

In [253]:
df_trades['GAIN'].sum()

-121.20000000000468

In [254]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
32,2022-05-02T04:00:00.000000000Z,0.75734,0.75778,0.75698,0.75774,0.757949,0.757992,-0.000042,0.000422,-1,29.6,-29.6
54,2022-05-03T02:00:00.000000000Z,0.76020,0.76090,0.76012,0.76070,0.758717,0.758653,0.000064,-0.000144,1,47.0,47.0
146,2022-05-08T22:00:00.000000000Z,0.76538,0.76611,0.76502,0.76540,0.766271,0.766467,-0.000196,0.000005,-1,26.4,-26.4
162,2022-05-09T14:00:00.000000000Z,0.76798,0.76857,0.76748,0.76804,0.766931,0.766887,0.000045,-0.000065,1,-27.9,-27.9
164,2022-05-09T16:00:00.000000000Z,0.76598,0.76609,0.76512,0.76525,0.766932,0.766934,-0.000002,0.000037,-1,2.2,-2.2


In [255]:
df_plot = df_ma.iloc[0:200].copy()

In [256]:
df_plot.shape

(200, 10)

In [257]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x= df_plot.time, 
    open= df_plot.mid_o, 
    high= df_plot.mid_h, 
    low= df_plot.mid_l, 
    close= df_plot.mid_c,
    line=dict(width=1),
    opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor='#CC2E3C',
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C')
)
for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name= col)
)
fig.update_layout(width=1000, 
                  height=400,
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e',
                  margin=dict(l=10,r=10,b=10,t=10),
                  font=dict(size=10,color='#e1e1e1')
)
fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,
    fixedrange=True,
    rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True,
)
fig.show()